# Synthethic data generation

You can use LLMs to generate training data for fine-tuning LLMs. As you migh expect, the generated data is not as good as the real data, but it can be useful to bootstrap your data for fine-tuning LLMs. In this tutorial, we provide easy and simple examples to generate synthetic data using LLMs, but given the architecture of `distilabel` it is easy to scale this to way more complex pipelines and larger workloads.

## Setup    

In [1]:
# install from develop because of small bug EvolQuality distilabel<1.2
%pip install "git+https://github.com/argilla-io/distilabel.git@develop#egg=distilabel[openai]" -qq 

Note: you may need to restart the kernel to use updated packages.


## Load dataset

We will use the dataset from our [Data is Better Together](https://github.com/huggingface/data-is-better-together). Data is Better Together is a collaboration between 🤗 Hugging Face, 🏓 Argilla, and the Open-Source ML community. We aim to empower the open-source community to build impactful datasets collectively.

This prompt ranking dataset was created by applying human evaluation in prompt, where roughly 400 people annotated human and synthehtic prompt to asses their quality on a scale from one to 5. 

In [10]:
from datasets import load_dataset

In [11]:
dataset = load_dataset("DIBT/10k_prompts_ranked")
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 10331
    })
})

In [12]:
dataset["train"][0]

{'prompt': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'quality': [{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb393a00',
   'value': '4',
   'status': 'submitted'},
  {'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e291889',
   'value': '4',
   'status': 'submitted'}],
 'metadata': '{"source": "ultrachat", "kind": "synthetic", "evolved_from": null}',
 'avg_rating': 5.0,
 'num_responses': 2,
 'agreement_ratio': 1.0,
 'raw_responses': [5, 5],
 'kind': 'synthetic',
 'cluster_description': 'Sustainable Packaging & Skin Care Products',
 'topic': 'Environmental Issues'}

## Load LLMs

We will now load a LLM integration within distilabel. For ease, we will use the `OpenAILLM`. Practically, there are two things to consider.

- You can customize use any propietary or open-source LLM you want to use for vendor lock-in and licensing reasons.
- Each LLM integration has its own arguments which are inherited from the original LLM provider.
- You might want to use different LLMs providers for different steps in the pipeline for diversity and quality reasons. 
- You might need to set `OPENAI_API_KEY` in your environment variables to use the LLMs.

In [6]:
from distilabel.llms import OpenAILLM

llm = OpenAILLM(model="gpt-4")

## Synthesizing Generations

We will now generate synthetic data using the LLM. We will use different prompt templates that were introduced and evaluated in various research papers. We call these `Tasks` within `distilabel`. Practically, there are several things to consider.

- `Tasks` are and are not exhaustive. You can create your own `Tasks` based on your use-case.
- `Tasks` are merely based on research papers and not always exact reproductions.

### Generate responses with `TextGeneration`

We don't always need to use complex verified prompts. We can also just go for basic chat completion. Practically, there are several things to consider.

- You might need to do some pre or post-processing to enure the data is formatted correctly.
- If working with chat data, you might want to use the `ChatGeneration` task instead.

In [3]:
from distilabel.steps.tasks import TextGeneration
 
text_generation = TextGeneration(name="text_generation", llm=llm)

[06/05/24 14:04:18] WARNING  WARNING:distilabel.step.text_generation:Step 'text_generation' hasn't      ]8;id=298338;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/steps/base.py\base.py]8;;\:]8;id=87738;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/steps/base.py#166\166]8;;\
                             received a pipeline, and it hasn't been created within a `Pipeline`                   
                             context. Please, use `with Pipeline() as pipeline:` and create the step               
                             within the context.                                                                   

In [4]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadHubDataset, KeepColumns
from distilabel.steps.tasks import TextGeneration

with Pipeline(name="text_generation") as pipeline_text_generation:
    load_hub_dataset = LoadHubDataset(
        name="load_dataset",
        repo_id="DIBT/10k_prompts_ranked",
        output_mappings={"prompt": "instruction"},
        split="train",
        num_examples=1
    )
    text_generation = TextGeneration(name="text_generation", llm=llm)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "generation",
        ],
    )
    load_hub_dataset >> text_generation >> keep_columns
    
distiset_text_generation = pipeline_text_generation.run()

[06/05/24 14:04:22] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=173904;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=914129;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/Users/davidberenstein/.cache/distilabel/pipelines/text_generation/44931             
                             5ac4554d4f71ad10f58a479a97f2bb1ff15/data'                                             

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=125336;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=886245;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#469\469]8;;\

[06/05/24 14:04:25] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 2/3                        ]8;id=753662;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=270176;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

[06/05/24 14:04:27] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 3/3                        ]8;id=76450;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=725549;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=719144;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=390237;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#487\487]8;;\

...

                    INFO     ['distilabel.step.text_generation'] 🏁 Finished running step              ]8;id=157645;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=460200;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\
                             'text_generation'                                                                     

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in 'keep_columns'  ]8;id=755081;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=184859;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#960\960]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending batch 0   ]8;id=191742;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=248246;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#998\998]8;;\
                             to output queue                                                                       

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step 'keep_columns'  ]8;id=510501;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=471952;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
distiset_text_generation["default"]["train"][0]

{'instruction': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'generation': "Here's a guide to making a homemade all-purpose cleaner. This guide doesn't include visuals or photographs as those can't be provided on this platform, but the steps are written clearly to guide you through the process effectively.\n\nIngredients:\n- 1 cup of white vinegar\n- 1 cup of water\n- 1/2 lemon juice (optional for extra disinfecting power and fresh scent)\n- 20-30 drops of essential oil such as lavender, tea tree, peppermint, or eucalyptus (optional for a pleasant aroma)\n\nSupplies:\n- A measuring cup\n- A spray bottle\n- A funnel"}

### Improve prompts with `SelfInstruct`

Based on the paper [Self-Instruct: Aligning LM with Self Generated Instructions](https://arxiv.org/abs/2212.10560). It relies on rewriting an instruction based on certain critaria that are deemed important. Practically, there are several things to consider.

- You might customize `criteria_for_query_generation` to improve the quality of the prompts to you domain.

In [5]:
### COMMENTED OUT BECAUSE IT REQUIRES RELOADING THE NOTEBOOK AND LLM ###
# from distilabel.steps.tasks import SelfInstruct
 
# self_instruct = SelfInstruct(name="self_instruct", llm=llm, num_instructions=1)
# self_instruct.load()
# self_instruct._template.render()

"# Task Description\nDevelop  user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model's textual capabilities.\n\n# Criteria for Queries\n\nWrite each query on a separate line and avoid using numbered lists or bullet points.\n\n# AI Application\n\n\n# Context\n\n\n# Output\n"

In [ ]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadHubDataset, KeepColumns
from distilabel.steps.tasks import SelfInstruct

with Pipeline(name="self_instruct") as pipeline_self_instruct:
    load_hub_dataset = LoadHubDataset(
        name="load_dataset",
        repo_id="DIBT/10k_prompts_ranked",
        output_mappings={"prompt": "input"},
        split="train",
        num_examples=1
    )
    self_instruct = SelfInstruct(name="self_instruct", llm=llm, num_instructions=1)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "input",
            "instructions",
        ],
    )
    load_hub_dataset >> self_instruct >> keep_columns
    
distiset_self_instruct = pipeline_self_instruct.run()

In [23]:
distiset_self_instruct["default"]["train"][0]

{'input': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'instructions': ['How can I create a safe and effective homemade all-purpose cleaner with common household ingredients?']}

### Improve responses with `EvolQuality`

Based on the paper [WizardLM: Empowering Large Language Models to Follow Complex Instructions](https://arxiv.org/pdf/2312.15685). It relies on an LLM to improve the quality response given an input based on different criteria. Practically, there are several things to consider.

- You might want to directly link this pipeline with the `TextGeneration` step.
- You might want to generate with `num_evolutions>1` so we directly have more than two options for preference data annotation

In [4]:
from distilabel.steps.tasks import EvolQuality

evol_quality = EvolQuality(name="evol_quality", llm=llm, num_evolutions=1)
evol_quality.mutation_templates

[06/05/24 14:38:27] WARNING  WARNING:distilabel.step.evol_quality:Step 'evol_quality' hasn't received a ]8;id=55601;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/steps/base.py\base.py]8;;\:]8;id=810939;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/steps/base.py#166\166]8;;\
                             pipeline, and it hasn't been created within a `Pipeline` context. Please,             
                             use `with Pipeline() as pipeline:` and create the step within the context.            

{'HELPFULNESS': "I want you to act as a Response Rewriter.\nYour goal is to enhance the quality of the response given by an AI assistant to the #Given Prompt# through rewriting.\nBut the rewritten prompt must be reasonable and must be understood and responded by humans.\nYour rewriting cannot omit the non-text parts such as the table and code in #Given Prompt# and #Given Response#. Also, please do not omit the input in #Given Prompt#.\n\nYou Should enhance the quality of the response using the following method: \nPlease make the Response more helpful to the user.\nYou should try your best not to make the #Rewritten Response# become verbose, #Rewritten Response# can only add 10 to 20 words into #Given Response#.\n'#Given Response#', '#Rewritten Response#', 'given response' and 'rewritten response' are not allowed to appear in #Rewritten Response#\n#Given Prompt#:\n<PROMPT>\n#Given Response#:\n<RESPONSE>\n#Rewritten Response#:\n",
 'RELEVANCE': "I want you to act as a Response Rewriter.\

In [7]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromDicts, KeepColumns
from distilabel.steps.tasks import EvolQuality

with Pipeline(name="evol_quality") as pipeline_evol_quality:
    load_data = LoadDataFromDicts(
        name="load_data",
        data=[
            {
                'instruction': "What did Leonarda Da Vinci focus on during his life?",
                'response': "He was an Italian scientist and engineer in the renaissance.",
            }
        ],
    )
    evol_quality = EvolQuality(name="evol_quality", llm=llm, num_evolutions=1)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "response",
            "evolved_response",
        ],
    )
    load_data >> evol_quality >> keep_columns

distiset_evol_quality = pipeline_evol_quality.run()

[06/05/24 14:39:51] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=647602;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=434657;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/Users/davidberenstein/.cache/distilabel/pipelines/evol_quality/08e62411             
                             01b70f873be5b07150639b18f384c321/data'                                                

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=589997;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=680405;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#469\469]8;;\

[06/05/24 14:39:54] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 3/3                        ]8;id=894616;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=832038;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=321694;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=259194;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#487\487]8;;\

[06/05/24 14:39:55] INFO     ['distilabel.step.load_data'] 🧬 Starting yielding batches from generator ]8;id=919329;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=741968;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#911\911]8;;\
                             step 'load_data'. Offset: 0                                                           

...

                    INFO     ['distilabel.step.evol_quality'] 🏁 Finished running step 'evol_quality'  ]8;id=174590;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=645606;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in 'keep_columns'  ]8;id=91255;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=246220;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#960\960]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending batch 0   ]8;id=323613;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=750524;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#998\998]8;;\
                             to output queue                                                                       

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step 'keep_columns'  ]8;id=579760;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=75568;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
distiset_evol_quality["default"]["train"][0]

{'instruction': 'What did Leonarda Da Vinci focus on during his life?',
 'response': 'He was an Italian scientist and engineer in the renaissance.',
 'evolved_response': 'Leonardo Da Vinci had a multifaceted focus throughout his life, which spanned interests in art, science, engineering, and the natural world.'}

## Synthesizing AI Feedback with LLMs as Judges 

We will now generate synthetic evaluations using the LLM. This once again relies on prompt templates and `Tasks` as with synthesizing generations. Practically, there are several things to consider.

- Only several LLMs can actually generate evaluations that align with human evaluations. Higher-end propietary models from companies like `mistral` and `OpenAI` are better at this. For open-source models, you might want to use `Prometheus 2.0`.
- You can use evaluate based on differnt aspects like helpfulness, relevance, and fluency, however, given the cost, an overall rating is usually sufficient.

### Absolute evaluation of reponses using `UltraFeedback`

Based on the paper [UltraFeedback: Boosting Language Models with High-quality Feedback](https://arxiv.org/abs/2310.01377). In this case, we will generate an absolute feedback score based on an overall rating. Practically, there are several things to consider.

- A single overall rating is usually sufficient but UltraFeedback also covers (Instruction-following, truthfullness, honesty, helpfulness).
- `PrometheusEval` and `Prometheus 2.0` are a good alternative for open-source models.

In [9]:
from distilabel.steps.tasks import UltraFeedback

ultra_feedback = UltraFeedback(name="ultra_feedback", llm=llm, aspect="overall-rating")
ultra_feedback._system_prompt

'Your role is to evaluate text quality based on given criteria.\nYou\'ll receive an instructional description ("Instruction") and {no_texts} text outputs ("Text").\nUnderstand and interpret instructions to evaluate effectively.\nProvide annotations for each text with a rating and rationale.\nThe {no_texts} texts given are independent, and should be evaluated separately.\n'

In [12]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromDicts, KeepColumns
from distilabel.steps.tasks import UltraFeedback

with Pipeline(name="ultra_feedback") as pipeline_ultra_feedback:
    load_data = LoadDataFromDicts(
        name="load_data",
        data=[
            {
                'instruction': "What did Leonarda Da Vinci focus on during his life?",
                'generations': [
                    "He was an Italian scientist and engineer in the renaissance.",
                    "He was a painter, sculptor, architect, and engineer.",
                ]
            }
        ],
    )
    ultra_feedback = UltraFeedback(name="ultra_feedback", llm=llm, aspect="overall-rating")
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "generations",
            "ratings",
            "rationales"
        ],
    )
    load_data >> ultra_feedback >> keep_columns
    
distiset_ultra_feedback = pipeline_ultra_feedback.run()

[06/05/24 15:03:15] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=525058;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=332371;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/Users/davidberenstein/.cache/distilabel/pipelines/ultra_feedback/a6fdea             
                             ccbcf7ffeaa4452f2c53ece2658648c5a3/data'                                              

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=481595;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=203951;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#469\469]8;;\

[06/05/24 15:03:18] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 3/3                        ]8;id=818933;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=386350;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=361139;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=202657;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#487\487]8;;\

[06/05/24 15:03:19] INFO     ['distilabel.step.load_data'] 🧬 Starting yielding batches from generator ]8;id=457624;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=487822;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#911\911]8;;\
                             step 'load_data'. Offset: 0                                                           

...

                    INFO     ['distilabel.step.ultra_feedback'] 🏁 Finished running step               ]8;id=373032;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=773587;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\
                             'ultra_feedback'                                                                      

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in 'keep_columns'  ]8;id=210480;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=166709;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#960\960]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending batch 0   ]8;id=475121;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=174751;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#998\998]8;;\
                             to output queue                                                                       

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step 'keep_columns'  ]8;id=743364;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=120804;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
distiset_ultra_feedback["default"]["train"][0]

{'instruction': 'What did Leonarda Da Vinci focus on during his life?',
 'generations': ['He was an Italian scientist and engineer in the renaissance.',
  'He was a painter, sculptor, architect, and engineer.'],
 'ratings': [3, 4],
 'rationales': ['The text correctly identifies Leonardo Da Vinci as a scientist and engineer during the renaissance, which is partially accurate. However, it fails to include essential aspects like his focus on painting, sculpture, and architecture, which is significant misinformation. Hence it partially follows the instruction.',
  "This text gives a more comprehensive answer, touching on Leonardo Da Vinci's roles as a painter, sculptor, architect, and engineer. However, it could be improved with a note on his contributions to science and inventiveness,"]}

### Relative evaluation (ranking) of responses using ranking `QualityScorer`

Based on the paper [WizardLM: Empowering Large Language Models to Follow Complex Instructions](https://arxiv.org/pdf/2312.15685). Practically, there are several things to consider.

- You might want use the `PairRM` model for predictive open source alternative to LLMs as relative judges.


In [17]:
### COMMENTED OUT BECAUSE IT REQUIRES RELOADING THE NOTEBOOK AND LLM ###
# from distilabel.steps.tasks import QualityScorer

# quality_scorer = QualityScorer(name="quality_scorer", llm=llm)
# quality_scorer.load()
# quality_scorer._template.render()

'Rank the following pair of instructions and responses according to their quality. Your evaluation should consider factors such as helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Score 1-0.\nScore each response from 1 to 0, with 1 reserved for responses that are already very well written and cannot be improved further. You should respond with the format:\n[1] Score: 1\n[2] Score: 2\n...\n#Question#: \n#Response List#:\n'

In [3]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromDicts, KeepColumns
from distilabel.steps.tasks import QualityScorer

with Pipeline(name="quality_scorer") as pipeline_quality_scorer:
    load_data = LoadDataFromDicts(
        name="load_data",
        data=[
            {
                'instruction': "What did Leonarda Da Vinci focus on during his life?",
                'responses': [
                    "He was an Italian scientist and engineer in the renaissance.",
                    "He was a painter, sculptor, architect, and engineer.",
                ]
            }
        ],
    )
    quality_scorer = QualityScorer(name="quality_scorer", llm=llm)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "responses",
            "scores",
        ],
    )
    load_data >> quality_scorer >> keep_columns

distiset_quality_scorer = pipeline_quality_scorer.run()

[06/05/24 15:39:42] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=525333;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=11003;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/Users/davidberenstein/.cache/distilabel/pipelines/quality_scorer/a70acf             
                             76907c8012403c03e551bac21479f726a2/data'                                              

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=171884;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=482659;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#469\469]8;;\

[06/05/24 15:39:44] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 3/3                        ]8;id=432404;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=849789;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=342759;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=806367;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#487\487]8;;\

[06/05/24 15:39:45] INFO     ['distilabel.step.load_data'] 🧬 Starting yielding batches from generator ]8;id=662106;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=153908;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#911\911]8;;\
                             step 'load_data'. Offset: 0                                                           

...

                    INFO     ['distilabel.step.quality_scorer'] 🏁 Finished running step               ]8;id=642072;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=987302;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\
                             'quality_scorer'                                                                      

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in 'keep_columns'  ]8;id=967199;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=206809;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#960\960]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending batch 0   ]8;id=295292;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=575483;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#998\998]8;;\
                             to output queue                                                                       

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step 'keep_columns'  ]8;id=816051;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=69966;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
distiset_quality_scorer["default"]["train"][0]

{'instruction': 'What did Leonarda Da Vinci focus on during his life?',
 'responses': ['He was an Italian scientist and engineer in the renaissance.',
  'He was a painter, sculptor, architect, and engineer.'],
 'scores': [1.0, 2.0]}

## A full pipeline 

We will now combine all the steps into a full pipeline. Practically, there are several things to consider.

- You can be as creative as you want with the pipeline.
- More complex pipelines generally require more computational resources, results are cached but it can still be expensive.

In [8]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadHubDataset, KeepColumns
from distilabel.steps.tasks import TextGeneration, EvolQuality, QualityScorer

with Pipeline(name="text_generation") as pipeline_complete:
    load_hub_dataset = LoadHubDataset(
        name="load_dataset",
        repo_id="DIBT/10k_prompts_ranked",
        output_mappings={"prompt": "instruction"},
        split="train",
        num_examples=4
    )
    text_generation = TextGeneration(name="text_generation", llm=llm)
    evol_quality = EvolQuality(name="evol_quality", llm=llm, num_evolutions=2, store_evolutions=True, input_mappings={"response": "generation"})
    quality_scorer = QualityScorer(name="quality_scorer", llm=llm, input_mappings={"responses": "evolved_responses"})
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "evolved_responses",
            "scores"
        ],
    )
    load_hub_dataset >> text_generation >> evol_quality >> quality_scorer >> keep_columns
    
    
distiset_complete = pipeline_complete.run()

[06/05/24 17:06:17] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=35012;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=313736;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/Users/davidberenstein/.cache/distilabel/pipelines/text_generation/f0899             
                             6f04836de25629e1873a295576ef48f8511/data'                                             

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=215993;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=853213;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#469\469]8;;\

[06/05/24 17:06:20] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 4/5                        ]8;id=84326;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=614509;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

[06/05/24 17:06:22] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 5/5                        ]8;id=883449;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=703753;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#483\483]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=981867;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=944977;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#487\487]8;;\

...

                    INFO     ['distilabel.step.quality_scorer'] 🏁 Finished running step               ]8;id=744413;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=740060;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\
                             'quality_scorer'                                                                      

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in 'keep_columns'  ]8;id=984370;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=372994;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#960\960]8;;\

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending batch 0   ]8;id=81532;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=937719;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#998\998]8;;\
                             to output queue                                                                       

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step 'keep_columns'  ]8;id=224577;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=263467;file:///Users/davidberenstein/Documents/programming/argilla/data-for-fine-tuning-llms/.env/lib/python3.10/site-packages/distilabel/pipeline/local.py#880\880]8;;\

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
distiset_complete["default"]["train"][:]

{'instruction': ['Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
  'Write a personal essay of at least 1000 words discussing how embracing vulnerability and authenticity has affected your life. Use specific examples from your own experiences to support your arguments and make sure to address the following questions:',
  'In this research, we aim to investigate how technology can moderate the correlation between knowledge management practices and the overall performance of an organization. This analysis will focus on specific technological tools and their impact on knowledge management in relation to various aspects of organizational performance. A

## Resources

Additinally, you can find more information on the following resources.

- [Distilabel 1.0 launch](https://argilla.io/blog/introducing-distilabel-1/)
- [Datasets on the Hugging Face hub](https://huggingface.co/datasets?other=distilabel&sort=trending)
- [Paper Implementations](https://distilabel.argilla.io/latest/sections/pipeline_samples/papers/)